Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv (либо его заархивированную версию salary-train.zip).

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk

In [2]:
data = pd.read_csv('salary-train.csv')
data_test = pd.read_csv('salary-test-mini.csv')

Проведите предобработку:

Приведите тексты к нижнему регистру (text.lower()).

Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. Для такой замены в строке text подходит следующий вызов: re.sub('[^a-zA-Z0-9]', ' ', text).

In [3]:
data['FullDescription'] = (data['FullDescription'].apply(lambda text: text.lower())).replace('[^a-zA-Z0-9]', ' ', regex = True)
data_test['FullDescription'] = (data_test['FullDescription'].apply(lambda text: text.lower())).replace('[^a-zA-Z0-9]', ' ', regex = True)

Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
tf_vec = TfidfVectorizer(min_df=5)
X_train = tf_vec.fit_transform(data['FullDescription'])

Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'. Код для этого был приведен выше.
Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.

In [6]:
data['LocationNormalized'].fillna('nan', inplace=True)
data['ContractTime'].fillna('nan', inplace=True)

In [7]:
from sklearn.feature_extraction import DictVectorizer

In [8]:
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data[['LocationNormalized', 'ContractTime']].to_dict('records'))

Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.

In [9]:
from scipy.sparse import hstack

In [10]:
X = hstack([X_train, X_train_categ])

Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная записана в столбце SalaryNormalized.

In [11]:
from sklearn.linear_model import Ridge

In [12]:
y_train = data['SalaryNormalized']
clf = Ridge(alpha=1, random_state=241)
clf.fit(X_train, y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

Постройте прогнозы для двух примеров из файла salary-test-mini.csv. Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.

In [34]:
X_test = tf_vec.transform(data_test['FullDescription'])
print '{:f} {:f}'.format(clf.predict(X_test)[0], clf.predict(X_test)[1])

56751.018289 38175.575897
